# Webscraper für die privaten Jobs auf www.stellenwerk-hamburg.de

In [64]:
!pip install requests
!pip install bs4
!pip install tqdm

  Using cached bs4-0.0.1-py3-none-any.whl


In [33]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import time
import csv

In [34]:
base_url = "https://www.stellenwerk-hamburg.de/jobboerse/privat"
query = {
    'keywords': '',
    'offer_type': 'job_ad_private',
    'job_category': 'All',
    'employment_type': 'All',
    'page': '0'
}

In [35]:
my_headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:91.0) Gecko/20100101 Firefox/91.0'}

## Aufruf der Webseite

In [36]:
r = requests.get(base_url, params = query, headers = my_headers)

In [37]:
print(r.request.url) # aufgerufene URL

https://www.stellenwerk-hamburg.de/jobboerse/privat?keywords=&offer_type=job_ad_private&job_category=All&employment_type=All&page=0


In [38]:
print(r.request.headers) # an den Webserver gesendete Anfragekopfezeile

{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:91.0) Gecko/20100101 Firefox/91.0', 'Accept-Encoding': 'gzip, deflate, br', 'Accept': '*/*', 'Connection': 'keep-alive'}


In [39]:
print(r.status_code) # War der Webseitenaufruf erfolgreich?
print(r.reason)

200
OK


## HTML-Code der Webseite speichern

In [40]:
html = r.text

In [41]:
bs = BeautifulSoup(html, 'html.parser')

## Gesuchte Informationen aus dem HTML-Code auslesen

In [42]:
job_ads = bs.find('div', class_ = 'JobSearch-content')

In [43]:
job_links = job_ads.find_all('a', class_ = 'Link')

Anzahl an Elementen (hier: Links zu den Stellenanzeigen) in der Liste *job_links*:

In [44]:
print(len(job_links))

20


In [45]:
for l in job_links:
    print(l['href'])

/jobboerse/bares-fuer-rares-pers-assistenz-priv-tags-24-30-hw-1549-1936-eu-hamburg-211210-507691
/jobboerse/hr-payroll-peronalabrechnung-duesseldorf-211210-507699
/jobboerse/nachhilfe-englisch-latein-und-mathe-hamburg-211210-507542
/jobboerse/persoenliche-assistentin-bei-den-landungsbruecken-hamburg-211210-507543
/jobboerse/suche-eine-nachhilfe-fuer-statistik-ii-hamburg-211210-507632
/jobboerse/teilnehmerinnen-fuer-online-studie-zu-gesundheitsrecherchen-gesucht-koeln-211210-507696
/jobboerse/wir-suchen-eine-zuverlaessige-haushalts-fee-hamburg-211210-507574
/jobboerse/freizeitbegleitung-rollstuhlfahrerin-alle-2wo-auf-rechnung-hamburg-211209-507425
/jobboerse/suche-hilfe-garten-und-haushalt-hamburg-211209-507455
/jobboerse/nachhilfe-statistik-hamburg-211208-507267
/jobboerse/nette-familie-sucht-putzhilfe-bahrenfeld-gross-flottbek-hamburg-211208-507099
/jobboerse/persoenliche-pflege-assistenz-hamburg-211208-507134
/jobboerse/php-developer-mit-hang-zum-wassersport-als-co-founder-gesucht-ha

In [46]:
rows = []
for link in job_links:
    link = 'https://www.stellenwerk-hamburg.de' + link['href']
    rows.append(link)

In [47]:
for row in rows:
    print(row)

https://www.stellenwerk-hamburg.de/jobboerse/bares-fuer-rares-pers-assistenz-priv-tags-24-30-hw-1549-1936-eu-hamburg-211210-507691
https://www.stellenwerk-hamburg.de/jobboerse/hr-payroll-peronalabrechnung-duesseldorf-211210-507699
https://www.stellenwerk-hamburg.de/jobboerse/nachhilfe-englisch-latein-und-mathe-hamburg-211210-507542
https://www.stellenwerk-hamburg.de/jobboerse/persoenliche-assistentin-bei-den-landungsbruecken-hamburg-211210-507543
https://www.stellenwerk-hamburg.de/jobboerse/suche-eine-nachhilfe-fuer-statistik-ii-hamburg-211210-507632
https://www.stellenwerk-hamburg.de/jobboerse/teilnehmerinnen-fuer-online-studie-zu-gesundheitsrecherchen-gesucht-koeln-211210-507696
https://www.stellenwerk-hamburg.de/jobboerse/wir-suchen-eine-zuverlaessige-haushalts-fee-hamburg-211210-507574
https://www.stellenwerk-hamburg.de/jobboerse/freizeitbegleitung-rollstuhlfahrerin-alle-2wo-auf-rechnung-hamburg-211209-507425
https://www.stellenwerk-hamburg.de/jobboerse/suche-hilfe-garten-und-haush

### Blättern

In [48]:
pages = bs.find('span', class_ = 'Pagination-text').get_text()
print(pages)
print(type(pages)) # Abfrage des Datentyps; hier: string (Zeichenkette)

Seite 1 von 7
<class 'str'>


In [49]:
page_max = int(pages.split(' ')[3])

In [50]:
print(page_max)
print(type(page_max)) # Abfrage des Datentyps; hier: integer (ganzzahliger Wert)

7
<class 'int'>


Nun kann diese Information genutzt werden, um zu blättern. Dazu wird eine *for*-Schleife in Kombination mit der Funktion `range()` verwendet. `range()` bestimmt, wie häufig die Schleife durchlaufen wird. Hier setzen wir *page_max* ein und addieren eine Eins hinzu, da `range()` standardmäßig den Stoppwert nicht mit einschließt (also nur bis *page_max* - 1 läuft). In jedem Schleifendurchlauf wird im *Query-String* der Wert für den Schlüssel *page* (s. oben unter Aufbau der URL des Stellenwerks) um eins erhöht. Im ersten Durchlauf ist page=0, im zweiten page=1, im dritten page=2 usw. Alle Links zu den Stellenanzeigen, die auf einer Seite zu finden sind, werden dann extrahiert und zu der Liste *all_links* hinzugefügt.<br>
Nach jedem Schleifendurchlauf stoppen wir mit `time.sleep(1)` das Programm für 1 Sekunde, bevor wir die nächste Seite aufrufen. Andernfalls würden wir mit zu vielen Anfragen in kurzer Zeit eventuell den Server überlasten. Dies würde schlimmstenfalls eine [Denial-of-Service-Attacke](https://www.bsi.bund.de/DE/Themen/Verbraucherinnen-und-Verbraucher/Cyber-Sicherheitslage/Methoden-der-Cyber-Kriminalitaet/DoS-Denial-of-Service/dos-denial-of-service_node.html) darstellen.

In [51]:
all_links = [] # Anlegen einer leeren Liste zur Aufnahme der Links
for page in tqdm(range(page_max + 1)): # tqdm vor range erzeugt einen Fortschrittsbalken 
    query["page"] = page # Seitenzahl wird in jedem Schleifendurchlauf um 1 erhöht
    r = requests.get(base_url, params = query) # Webseite aufrufen
    html = r.text # HTML-Code der Webseite speichern
    bs = BeautifulSoup(html, 'html.parser') # HTML-Code an BeautifulSoup übergeben
    job_ads = bs.find('div', class_ = 'JobSearch-content') # Element mit Links zu den Stellenanzeigen extrahieren
    job_links = job_ads.find_all('a', class_ = 'Link') # aus dieser Spalte alle Links zu den Anzeigen extrahieren
    for link in job_links: # Protokoll und Hostname jedem Link voranstellen
        link = 'https://www.stellenwerk-hamburg.de' + link['href']
        all_links.append(link) # Links der Liste all_links hinzufügen
    time.sleep(1) # eine Sekunde Verzögerung nach jedem Seitenaufruf (friendly scraping)

  0%|          | 0/8 [00:00<?, ?it/s]

Wie viele Elemente hat die Liste *all_links* nach dem durch alle Seiten geblättert wurde? Diese Anzahl entspricht der Anzahl an Stellenangeboten zum Scraping-Zeitpunkt:

In [52]:
print(len(all_links))

127


Nun liegt eine Liste mit allen Links zu den Stellenangeboten vor. Jetzt muss noch bestimmt werden, welche Informationen zu den jeweiligen Stellenangeboten extrahiert werden sollen. Dazu ruft man beispielhaft eine oder mehrere Stellenanzeigen aus der Liste *all_links* auf untersucht den HTML-Quellcode mit den Entwicklerwerkzeugen. So können alle HTML-Elemente identifiziert werden, in denen die Job-spezifischen Informationen zu finden sind.

In [24]:
url = "https://www.stellenwerk-hamburg.de/jobboerse/bares-fuer-rares-pers-assistenz-priv-tags-24-30-hw-1549-1936-eu-hamburg-211210-507691"
r = requests.get(url)
html = r.text
bs = BeautifulSoup(html, 'html.parser')
print(r.status_code) # War der Webseitenaufruf erfolgreich?
print(r.reason)

200
OK


### Allgemeine Informationen

Allgemeinere Informationen wie Job-Bezeichnung, Datum und ID der Anzeige sowie der Anzeigetyp finden sich im oberen Bereich der Seite und lassen sich wie folgt aus dem Quellcode herauslesen.
Die Informationen zu *ad_date*, *ad_id* und *ad_type* befinden sich in einem [*ul*-Tag](https://www.w3schools.com/tags/tag_ul.asp), mit dem ungeordnete Listen angelegt werden. Die Listeeinträge starten jeweils mit einem *li*-Tag. Da *ad_date* die erste *li*-Instanz ist, kann diese Information einfach mit `.find()` entnommen werden. Der Eintrag enthält die Zeichenkette "Online seit DD.MM.YYYY". Diese wird mit Hilfe der Methode `.split()` am Leerzeichen (' ') in ihre Komponenten zerlegt und dann das dritte Element (DD.MM.YYYY) extrahiert.
Bei *ad_id* und *ad_type* kommt die Funktion `.select()` von *BeautifulSoup* zum Einsatz, der man einen [CSS-Selektor](https://www.w3schools.com/cssref/css_selectors.asp) übergeben kann. Die Informationen
Rechtsklick Kontextmenü 

In [25]:
job_title = bs.find('h1', class_ = 'JobHead-title').get_text()
ad_date = bs.find('li', class_ = 'JobHead-additionalInfoItem').get_text().split(' ')[2]
ad_id = bs.select('li.JobHead-additionalInfoItem:nth-child(2)')[0].get_text()
ad_type = bs.select('li.JobHead-additionalInfoItem:nth-child(3)')[0].get_text()

### Anzeigendaten

Informationen wie Art und Zeitraum der Beschäftigung, Vergütung sowie Bewerbungslink oder Bewerbungs-E-Mail (es gibt nur eins von beiden) finden sich im unteren Drittel der Stellenanzeigen.<br>
Hier werden das erste Mal mit [*try* und *except*](https://www.w3schools.com/python/python_try_except.asp) Ausnahmen behandelt:<br>
+ Nicht jede Stellenanzeige weist eine Vergütung aus
+ Wenn ein Bewerbunslink angegeben wurde, fehlt die Bewerbungs-E-Mail und vice versa, da es auf der Webseite des Stellenwerks sich ausschließende Alternativen sind.

Ein Exraktionsversuch bei fehlenden Informationen würde zu einer Fehlermeldung und zu einem Programmabbruch führen. Für so einen Fall wird im *except*-Block explizit der Wert *None* vergeben. Durch diese Konstruktion wird ein Programmabbruch verhindert:

In [31]:
job_type = bs.find(string = "Art der Beschäftigung").find_next('dd').get_text() # Suche nach dem Tag mit Inhalt "Art der Beschäftigung", dann das folgende dd-Tag wählen
job_duration = bs.find(string = "Zeitraum der Beschäftigung").find_next('dd').get_text()
try:
    job_wage = bs.find(string = "Vergütung").find_next('dd').get_text()
except:
    job_wage = None
try:
    app_link = bs.find(string = "Bewerbungslink").find_next('dd').find_next('a', class_ = 'Link')['href']
except:
    app_link = None
try:
    app_email = bs.find(string = "Bewerbungs-E-Mail").find_next('a', class_ = 'IconText-mail IconText')['href']
except:
    app_email = None

### Beschreibung und Anforderungsprofil der Stelle

Die Beschreibung der Stelle und das Anforderungsprofil sind die zentralen stellenbezogenen Informationen und finden sich in zwei seperaten Textfeldern im mittleren Teil der Seite:

In [28]:
job_descrip = bs.find_all('div', class_ = 'RichText')[0].get_text()
job_profile = bs.find_all('div', class_ = 'RichText')[1].get_text()

### Firmenkontaktdaten

Die Firmenkontaktdaten stehen am Ende der Seite und beinhalten Informationen wie Firmenname, Adresse und Kontaktperson:

In [30]:
loc = bs.find(string = "Standort").find_all_next('dd', limit = 2)
location = loc[0].get_text()+ ", " + loc[1].get_text()
contact = bs.find(string = "Kontaktperson").find_next('dd').get_text()

## Schleife über alle Stellenanzeigen

Nun wird alles kombiniert. Die Informationen (allgemeine Informationen, Anzeigendaten, Beschreibung und Anforderungsprofil der Stelle, Firmeninformationen) müssen nun für *alle* Stellenanzeigen extrahiert werden. Mit Hilfe einer *for*-Schleife werden dafür alle Links aus *all_links* nacheinander aufgerufen, der jeweilige HTML-Quellcode abgerufen, aus diesem die relevanten Informationen zu jeder Stellenanzeige extrahiert und in einem Python-Dictionary (Schlüssel-Wert-Paare) names *row* gespeichert. Diese Dictionaries werden dann ein eine Liste namens *rows* übertragen, d.h. die Liste hat so viele Elemente in Form von Dictionaries, wie es Stellenanzeigen gibt. Die *try*-und-*except*-Blöcke verhindern den Programm-Abbruch falls in einzelnen Stellenanzeigen manche Informationen nicht vorhanden sind und somit nicht ausgelesen werden können.

In [53]:
rows = []
for link in tqdm(all_links): # tqdm vor der iterierbaren Liste 'all_links' erzeugt einen Fortschrittsbalken 
    r = requests.get(link, headers = my_headers) # Webseite aufrufen
    html = r.text # HTML-Code der Webseite speichern
    bs = BeautifulSoup(html, 'html.parser') # HTML-Code an BeautifulSoup übergeben
    try: # alle benötigen Informationen aus dem HTML-Code extrahieren
        job_title = bs.find('h1', class_ = 'JobHead-title').get_text()
    except:
        job_title = None
    try:
        ad_date = bs.find('li', class_ = 'JobHead-additionalInfoItem').get_text().split(' ')[2]
    except:
        ad_date = None
    try:    
        ad_id = bs.select('li.JobHead-additionalInfoItem:nth-child(2)')[0].get_text()
    except:
        ad_id = None
    try:
        ad_type = bs.select('li.JobHead-additionalInfoItem:nth-child(3)')[0].get_text()
    except:
        ad_type = None
    try:
        job_type = bs.find(string = "Art der Beschäftigung").find_next('dd').get_text() # Suche nach dem Tag mit Inhalt "Art der Beschäftigung", dann das folgende dd-Tag wählen
    except:
        job_type = None
    try:
        job_duration = bs.find(string = "Zeitraum der Beschäftigung").find_next('dd').get_text()
    except:
        job_duration = None
    try:
        job_wage = bs.find(string = "Vergütung").find_next('dd').get_text()
    except:
        job_wage = None
    try:
        app_link = bs.find(string = "Bewerbungslink").find_next('dd').find_next('a', class_ = 'Link')['href']
    except:
        app_link = None
    try:
        app_email = bs.find(string = "Bewerbungs-E-Mail").find_next('a', class_ = 'IconText-mail IconText')['href']
    except:
        app_email = None
    try:
        loc = bs.find(string = "Standort").find_all_next('dd', limit = 2)
        location = loc[0].get_text()+ ", " + loc[1].get_text()
    except:
        loc, location = None, None
    try:
        contact = bs.find(string = "Kontaktperson").find_next('dd').get_text()
    except:
        contact = None
    try:
        job_descrip = bs.find_all('div', class_ = 'RichText')[0].get_text()
    except:
        job_descrip = None
    try:
        job_profile = bs.find_all('div', class_ = 'RichText')[1].get_text()
    except:
        job_profile = None
    row = { # alle extrahierten Informationen in ein Dictionary schreiben
    'job_title': job_title,
    'ad_date': ad_date,
    'ad_id': ad_id,
    'ad_type': ad_type,
    'job_type': job_type,
    'job_duration': job_duration,
    'job_wage': job_wage,
    'app_link': app_link,
    'app_email': app_email,
    'location': location,
    'contact': contact,
    'job_descrip': job_descrip,
    'job_profile': job_profile,
    'link': link
    }
    rows.append(row) # Dictionary names row wird der Liste namens rows angehängt
    time.sleep(1) # eine Sekunde Verzögerung nach jedem Seitenaufruf (friendly scraping)

  0%|          | 0/127 [00:00<?, ?it/s]

## Extrahierte Daten in CSV-Datei schreiben

Die extrahierten Daten, die in der Liste *rows* gespeichert sind, werden abschließend in eine rechteckige Datenmatrix geschrieben. Jede Stellenanzeige (Merkmalsträger) bildet eine Zeile. Die Merkmale der Stellenanzeigen bilden die Spalten:

In [56]:
file = 'daten/job_ads_privat.csv'
with open(file, mode = 'w', encoding = 'utf-8', newline = '') as f:
    col_names = ['job_title', 'ad_date', 'ad_id', 'ad_type', 'job_type', 'job_duration', 'job_wage', 'app_link', 'app_email', 'location', 'contact', 'job_descrip', 'job_profile', 'link']
    writer = csv.DictWriter(f, fieldnames = col_names)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)